In [6]:
import numpy as np
import pandas as pd
import json
import math
from pathlib import Path

# โมเดล
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Keras RNN
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam


ImportError: Traceback (most recent call last):
  File "c:\VS CODE\.conda\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [2]:
pip install xgboost

   ---------------------------------------- 0.0/56.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/56.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/56.8 MB 162.5 kB/s eta 0:05:50
   ---------------------------------------- 0.0/56.8 MB 217.9 kB/s eta 0:04:21
   ---------------------------------------- 0.1/56.8 MB 357.2 kB/s eta 0:02:39
   ---------------------------------------- 0.1/56.8 MB 607.9 kB/s eta 0:01:34
   ---------------------------------------- 0.3/56.8 MB 962.4 kB/s eta 0:00:59
   ---------------------------------------- 0.4/56.8 MB 1.3 MB/s eta 0:00:44
   ---------------------------------------- 0.5/56.8 MB 1.5 MB/s eta 0:00:39
   ---------------------------------------- 0.6/56.8 MB 1.6 MB/s eta 0:00:36
    --------------------------------------- 0.7/56.8 MB 1.7 MB/s eta 0:00:34
    --------------------------------------- 0.9/56.8 MB 1.8 MB/s eta 0:00:31
    --------------------------------------- 1.0/56.8 MB 1.9 MB/s eta 0:00:29
   

In [4]:
pip install tensorflow

  Using cached tensorflow-2.20.0-cp311-cp311-win_amd64.whl.metadata (4.6 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-6.32.1-cp310-abi3-win_amd64.whl.metadata (593 bytes)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached tensorboard-2.20.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.11.3-py3-none-any.whl.metadata (5.9 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached namex-0.1.0-py3-none-any.whl.metadata (322 bytes)
  Using cached markd

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.37.1 requires protobuf<6,>=3.20, but you have protobuf 6.32.1 which is incompatible.


In [8]:
# โหลดข้อมูลจาก Sheet2 ของไฟล์ Excel
df = pd.read_excel(r"D:\Projeck\new data.xlsx", sheet_name="Sheet2")

# แสดง 5 แถวแรก
print(df.head())

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [48]:
import pandas as pd
import numpy as np

# ----------------- โหลดข้อมูลจาก Sheet2 -----------------
df = pd.read_excel(r"D:\Projeck\new data.xlsx", sheet_name="Sheet2")

# ----------------- ลบค่า '-' และแปลงเป็นตัวเลข -----------------
df = df.replace('-', np.nan)
df = df.apply(pd.to_numeric, errors='ignore')
df = df.dropna()

# ----------------- แปลง Date เป็น datetime -----------------
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# ----------------- แยกวัน เดือน ปี -----------------
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

# ----------------- ลบคอลัมน์ Date -----------------
df = df.drop(columns=['Date','Year'])

# ----------------- แสดงตัวอย่าง -----------------
print(df.head())


   Showalter  Lifted   SWEAT     K  Cross  Vertical  TT Totals   CAPE    CIN  \
0      13.29   16.28  244.92 -27.1    5.7      19.7       25.4   0.00   0.00   
1      10.78   11.77  127.18 -22.1   10.3      19.3       29.6   0.00   0.00   
2       3.53    4.54  362.30 -14.3   19.3      19.7       39.0  15.68 -46.73   
3       4.66    5.24  200.40 -18.0   18.8      19.3       38.1   0.00 -50.25   
4       7.09    8.06  170.99  -0.7   15.3      18.3       33.6   0.00   0.00   

    BRN  LCL Temp   LCL P  LCL P.1   MML θ  MML q  Thickness   PWAT  dBZ  \
0  0.00    276.14  783.20   314.16  296.13   6.16     5749.0  12.64  0.0   
1  0.00    281.45  839.90   319.96  295.87   8.30     5746.0  16.33  0.0   
2  0.86    288.03  903.90   331.01  296.49  11.91     5729.0  25.68  0.0   
3  0.00    285.99  879.04   327.88  296.74  10.71     5734.0  22.45  0.0   
4  0.00    285.51  868.80   327.84  297.24  10.51     5733.0  22.85  0.0   

   Month  Day  
0      2    1  
1      2    2  
2      2    3 

C:\Users\HP\AppData\Local\Temp\ipykernel_3316\1432247620.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace('-', np.nan)
C:\Users\HP\AppData\Local\Temp\ipykernel_3316\1432247620.py:9: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')


In [49]:
df

,Showalter,Lifted,SWEAT,K,Cross,Vertical,TT Totals,CAPE,CIN,BRN,LCL Temp,LCL P,LCL P.1,MML θ,MML q,Thickness,PWAT,dBZ,Month,Day
0,13.29,16.28,244.92,-27.1,5.7,19.7,25.4,0.00,0.00,0.00,276.14,783.20,314.16,296.13,6.16,5749.0,12.64,0.0,2,1
1,10.78,11.77,127.18,-22.1,10.3,19.3,29.6,0.00,0.00,0.00,281.45,839.90,319.96,295.87,8.30,5746.0,16.33,0.0,2,2
2,3.53,4.54,362.30,-14.3,19.3,19.7,39.0,15.68,-46.73,0.86,288.03,903.90,331.01,296.49,11.91,5729.0,25.68,0.0,2,3
3,4.66,5.24,200.40,-18.0,18.8,19.3,38.1,0.00,-50.25,0.00,285.99,879.04,327.88,296.74,10.71,5734.0,22.45,0.0,2,4
4,7.09,8.06,170.99,-0.7,15.3,18.3,33.6,0.00,0.00,0.00,285.51,868.80,327.84,297.24,10.51,5733.0,22.85,0.0,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,-4.55,-5.04,352.01,40.0,25.5,30.3,55.8,318.19,-95.93,3.38,289.19,896.00,336.27,298.43,13.00,5695.0,38.11,36.5,2,8
368,0.55,2.57,162.58,2.0,23.0,23.7,46.7,6.52,-108.75,0.17,284.68,877.22,324.07,295.55,9.87,5636.0,22.42,31.5,2,9
369,8.44,9.02,142.21,-4.7,13.7,19.7,33.4,0.00,0.00,0.00,284.29,880.49,322.46,294.84,9.55,5677.0,19.51,44.0,2,10
370,10.57,10.84,103.01,-18.9,10.1,20.1,30.2,0.00,0.00,0.00,283.37,871.98,320.92,294.70,9.08,5721.0,18.15,39.0,2,11


In [50]:
# ----------------- แยก X, y -----------------
X = df.drop(['Date'], axis=1, errors='ignore')
y = df['dBZ']

In [51]:
X

,Showalter,Lifted,SWEAT,K,Cross,Vertical,TT Totals,CAPE,CIN,BRN,LCL Temp,LCL P,LCL P.1,MML θ,MML q,Thickness,PWAT,dBZ,Month,Day
0,13.29,16.28,244.92,-27.1,5.7,19.7,25.4,0.00,0.00,0.00,276.14,783.20,314.16,296.13,6.16,5749.0,12.64,0.0,2,1
1,10.78,11.77,127.18,-22.1,10.3,19.3,29.6,0.00,0.00,0.00,281.45,839.90,319.96,295.87,8.30,5746.0,16.33,0.0,2,2
2,3.53,4.54,362.30,-14.3,19.3,19.7,39.0,15.68,-46.73,0.86,288.03,903.90,331.01,296.49,11.91,5729.0,25.68,0.0,2,3
3,4.66,5.24,200.40,-18.0,18.8,19.3,38.1,0.00,-50.25,0.00,285.99,879.04,327.88,296.74,10.71,5734.0,22.45,0.0,2,4
4,7.09,8.06,170.99,-0.7,15.3,18.3,33.6,0.00,0.00,0.00,285.51,868.80,327.84,297.24,10.51,5733.0,22.85,0.0,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,-4.55,-5.04,352.01,40.0,25.5,30.3,55.8,318.19,-95.93,3.38,289.19,896.00,336.27,298.43,13.00,5695.0,38.11,36.5,2,8
368,0.55,2.57,162.58,2.0,23.0,23.7,46.7,6.52,-108.75,0.17,284.68,877.22,324.07,295.55,9.87,5636.0,22.42,31.5,2,9
369,8.44,9.02,142.21,-4.7,13.7,19.7,33.4,0.00,0.00,0.00,284.29,880.49,322.46,294.84,9.55,5677.0,19.51,44.0,2,10
370,10.57,10.84,103.01,-18.9,10.1,20.1,30.2,0.00,0.00,0.00,283.37,871.98,320.92,294.70,9.08,5721.0,18.15,39.0,2,11


In [52]:
# ----------------- คำนวณ Correlation -----------------
corr = X.corrwith(y)


# เลือกเก็บและตัดออก
selected_features = corr[abs(corr) >= 0.05].index.tolist()
removed_features = corr[abs(corr) < 0.05].index.tolist()

# ----------------- สร้าง X ใหม่ -----------------
X_selected = X[selected_features]

In [53]:
# ----------------- แสดงผล -----------------
print("✅ คอลัมน์ที่ถูกเก็บไว้ (Correlation >= 0.05):")
print(selected_features)

print("\n❌ คอลัมน์ที่ถูกตัดออก (Correlation < 0.05):")
print(removed_features)                      

✅ คอลัมน์ที่ถูกเก็บไว้ (Correlation >= 0.05):
['Showalter', 'Lifted', 'SWEAT', 'K', 'Cross', 'Vertical', 'TT Totals', 'CAPE', 'CIN', 'LCL Temp', 'LCL P', 'LCL P.1', 'MML θ', 'MML q', 'Thickness', 'PWAT', 'dBZ', 'Month', 'Day']

❌ คอลัมน์ที่ถูกตัดออก (Correlation < 0.05):
['BRN']


In [58]:
y

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
367    36.5
368    31.5
369    44.0
370    39.0
371    39.0
Name: dBZ, Length: 303, dtype: float64

In [60]:
# บังคับให้ y เป็น Series
if isinstance(y, pd.DataFrame):
    y = y.squeeze()

# เช็คว่ามี class มากกว่า 1
if y.nunique() > 1:
    X_train, X_test, y_train, y_test = train_test_split(
        X_selected, y, test_size=0.2, random_state=42
    )
else:
    # ถ้ามี class เดียว ใช้แบบไม่ stratify
    X_train, X_test, y_train, y_test = train_test_split(
        X_selected, y, test_size=0.2, random_state=42
    )


In [62]:
# แปลง y เป็น binary class
threshold = 50  # ตัวอย่าง: ถ้าฝนเกิน 50mm ถือว่า "พายุฝน"
y_class = (y >= threshold).astype(int)

# แล้วใช้ y_class แทน y
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y_class, test_size=0.2, random_state=42, stratify=y_class
)


In [63]:
# ----------------- Import Libraries -----------------
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb

# ----------------- สร้างโมเดล -----------------
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "XGBoost": xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42),
    "Neural Network (MLP)": MLPClassifier(hidden_layer_sizes=(100,50), max_iter=500, random_state=42)
}

# ----------------- เทรนและประเมินผล -----------------
results = {}

for name, model in models.items():
    model.fit(X_train, y_train)             # เทรน
    y_pred = model.predict(X_test)          # ทำนาย
    acc = accuracy_score(y_test, y_pred)    # คำนวณ Accuracy
    results[name] = acc

    print("="*50)
    print(f"ผลลัพธ์จาก {name}")
    print("Accuracy:", acc)
    print(classification_report(y_test, y_pred))

# ----------------- แสดงผลรวม -----------------
print("\n📊 สรุปความแม่นยำของแต่ละโมเดล:")
for name, acc in results.items():
    print(f"{name}: {acc:.4f}")


ผลลัพธ์จาก Logistic Regression
Accuracy: 0.9836065573770492
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        59
           1       1.00      0.50      0.67         2

    accuracy                           0.98        61
   macro avg       0.99      0.75      0.83        61
weighted avg       0.98      0.98      0.98        61

ผลลัพธ์จาก Random Forest
Accuracy: 0.9836065573770492
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        59
           1       1.00      0.50      0.67         2

    accuracy                           0.98        61
   macro avg       0.99      0.75      0.83        61
weighted avg       0.98      0.98      0.98        61

ผลลัพธ์จาก XGBoost
Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        59
           1       1.00      1.00      1.00         2

    accuracy                       

C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\xgboost\training.py:183: UserWarning: [14:30:43] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and bei

In [67]:
# ----------------- Import Libraries -----------------
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from imblearn.over_sampling import SMOTE
import xgboost as xgb

# ----------------- แบ่ง Train/Test -----------------
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y_class, test_size=0.2, random_state=42, stratify=y_class
)

# ----------------- ใช้ SMOTE -----------------
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

print("ก่อนใช้ SMOTE:", y_train.value_counts().to_dict())
print("หลังใช้ SMOTE:", y_train_res.value_counts().to_dict())

# ----------------- สร้างโมเดล -----------------
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "XGBoost": xgb.XGBClassifier(eval_metric="logloss", random_state=42),
    "Neural Network (MLP)": MLPClassifier(hidden_layer_sizes=(100,50), max_iter=500, random_state=42)
}

# ----------------- เทรนและประเมินผล -----------------
results = {}

for name, model in models.items():
    model.fit(X_train_res, y_train_res)            # เทรนด้วยข้อมูล balance
    y_pred = model.predict(X_test)                 # ทำนาย
    y_proba = model.predict_proba(X_test)[:,1]     # สำหรับคำนวณ AUC

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    auc = roc_auc_score(y_test, y_proba)

    results[name] = {"Accuracy": acc, "F1-score": f1, "ROC-AUC": auc}

    print("="*50)
    print(f"ผลลัพธ์จาก {name}")
    print("Accuracy:", acc)
    print("F1-score:", f1)
    print("ROC-AUC:", auc)
    print(classification_report(y_test, y_pred, zero_division=0))

# ----------------- สรุปผลรวม -----------------
print("\n📊 สรุปประสิทธิภาพของแต่ละโมเดล:")
results_df = pd.DataFrame(results).T
print(results_df)

# ----------------- เลือกโมเดลที่ดีที่สุด (ตาม ROC-AUC) -----------------
best_model_name = results_df["ROC-AUC"].idxmax()
print(f"\n🏆 โมเดลที่ดีที่สุดคือ: {best_model_name}")

best_model = models[best_model_name]

# ----------------- บันทึกโมเดล -----------------
import joblib
joblib.dump(best_model, "storm_model.pkl")
print("✅ บันทึกโมเดลแล้ว: storm_model.pkl")


ก่อนใช้ SMOTE: {0: 234, 1: 8}
หลังใช้ SMOTE: {0: 234, 1: 234}
ผลลัพธ์จาก Logistic Regression
Accuracy: 0.9836065573770492
F1-score: 0.6666666666666666
ROC-AUC: 0.923728813559322
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        59
           1       1.00      0.50      0.67         2

    accuracy                           0.98        61
   macro avg       0.99      0.75      0.83        61
weighted avg       0.98      0.98      0.98        61

ผลลัพธ์จาก Random Forest
Accuracy: 0.9836065573770492
F1-score: 0.6666666666666666
ROC-AUC: 1.0
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        59
           1       1.00      0.50      0.67         2

    accuracy                           0.98        61
   macro avg       0.99      0.75      0.83        61
weighted avg       0.98      0.98      0.98        61

ผลลัพธ์จาก XGBoost
Accuracy: 1.0
F1-score: 1.0
ROC-AUC: 1.0
            

In [66]:
pip install imblearn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\HP\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
